In [3]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
import PIL
import rasterio

# Load VGG16 model + higher level layers
base_model = VGG16()
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_data = img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features.flatten()

def extract_coordinates_from_tif(tif_path):
    with rasterio.open(tif_path) as src:
        bounds = src.bounds
        # Calculate the center coordinates
        x_center = (bounds.left + bounds.right) / 2
        y_center = (bounds.top + bounds.bottom) / 2
        return x_center, y_center

# Assuming your images are in a directory called 'images'
# image_dir = 'h:\\Yehmh\\ZF\\202307\\5m_5m_51_rgb'
image_dir = '\\Yehmh\\FuShan\\Fushan_seg_images'
feature_list = []
image_names = []
coordinates_list = []

for img_file in os.listdir(image_dir):
    if img_file.lower().endswith('.tif'):
        print(img_file)
        img_path = os.path.join(image_dir, img_file)
        features = extract_features(img_path, model)
        feature_list.append(features)

        image_names.append(img_file)
        
        x, y = extract_coordinates_from_tif(img_path)
        coordinates_list.append((x, y))

features_array = np.array(feature_list)

# Dimensionality Reduction
pca = PCA(n_components=50)
pca_result = pca.fit_transform(features_array)

# Clustering
kmeans = KMeans(n_clusters=10)
kmeans.fit(pca_result)
labels = kmeans.labels_

# Save results to CSV
results_df = pd.DataFrame({
    'image_name': image_names,
    'x': [coord[0] for coord in coordinates_list],
    'y': [coord[1] for coord in coordinates_list],
    'cluster': labels
})

results_df.to_csv('\\Yehmh\\FuShan\\clustering_results.csv', index=False)

print("Clustering results saved to clustering_results.csv")


cropped_0.tif
1/1 [==============================] - 0s 64ms/step
cropped_1.tif
1/1 [==============================] - 0s 17ms/step
cropped_2.tif
1/1 [==============================] - 0s 15ms/step
cropped_3.tif
1/1 [==============================] - 0s 17ms/step
cropped_4.tif
1/1 [==============================] - 0s 17ms/step
cropped_5.tif
1/1 [==============================] - 0s 1ms/step
cropped_6.tif
1/1 [==============================] - 0s 16ms/step
cropped_7.tif
1/1 [==============================] - 0s 16ms/step
cropped_8.tif
1/1 [==============================] - 0s 18ms/step
cropped_9.tif
1/1 [==============================] - 0s 17ms/step
cropped_10.tif
1/1 [==============================] - 0s 19ms/step
cropped_11.tif
1/1 [==============================] - 0s 16ms/step
cropped_12.tif
1/1 [==============================] - 0s 17ms/step
cropped_13.tif
1/1 [==============================] - 0s 17ms/step
cropped_14.tif
1/1 [==============================] - 0s 18ms/step
croppe

In [2]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
from PIL import Image
import rasterio

# Load VGG16 model + higher level layers
base_model = VGG16()
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def extract_features(img_path, model):
     # Load the image
    img = Image.open(img_path)
    
    # Get the dimensions of the image
    width, height = img.size
    
    # Check if the image is smaller than the target size
    if width < 244 or height < 244:
        print(f"Image at {img_path} is smaller than the target size and cannot be cropped to 244x244")
        return None
    
    # Calculate the coordinates for the center crop
    left = (width - 244) // 2
    top = (height - 244) // 2
    right = left + 244
    bottom = top + 244
    
    # Crop the image
    img_cropped = img.crop((left, top, right, bottom))
    img_resized = img_cropped.resize((224, 224))
    img_array = img_to_array(img_resized)
    
    # Expand dimensions to match the input shape of VGG16 (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array[:,:,:,:3]
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    
    return features.flatten()

def extract_coordinates_from_tif(tif_path):
    with rasterio.open(tif_path) as src:
        bounds = src.bounds
        # Calculate the center coordinates
        x_center = (bounds.left + bounds.right) / 2
        y_center = (bounds.top + bounds.bottom) / 2
        return x_center, y_center

# Assuming your images are in a directory called 'images'
# image_dir = 'h:\\Yehmh\\ZF\\202307\\5m_5m_51_rgb'
image_dir = '\\Yehmh\\FuShan\\Fushan_seg_images'
feature_list = []
image_names = []
coordinates_list = []

for img_file in os.listdir(image_dir):
    if img_file.lower().endswith('.tif'):
        print(img_file)
        img_path = os.path.join(image_dir, img_file)
        features = extract_features(img_path, model)
        if features is not None:
            feature_list.append(features)
            image_names.append(img_file)        
            x, y = extract_coordinates_from_tif(img_path)
            coordinates_list.append((x, y))

features_array = np.array(feature_list)

# Dimensionality Reduction
pca = PCA(n_components=50)
pca_result = pca.fit_transform(features_array)

# Clustering
kmeans = KMeans(n_clusters=10)
kmeans.fit(pca_result)
labels = kmeans.labels_

# Save results to CSV
results_df = pd.DataFrame({
    'image_name': image_names,
    'x': [coord[0] for coord in coordinates_list],
    'y': [coord[1] for coord in coordinates_list],
    'cluster': labels
})

results_df.to_csv('\\Yehmh\\FuShan\\clustering_results_crop.csv', index=False)

print("Clustering results saved to clustering_results.csv")


cropped_0.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_0.tif is smaller than the target size and cannot be cropped to 244x244
cropped_1.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_1.tif is smaller than the target size and cannot be cropped to 244x244
cropped_2.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_2.tif is smaller than the target size and cannot be cropped to 244x244
cropped_3.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_3.tif is smaller than the target size and cannot be cropped to 244x244
cropped_4.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_4.tif is smaller than the target size and cannot be cropped to 244x244
cropped_5.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_5.tif is smaller than the target size and cannot be cropped to 244x244
cropped_6.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped_6.tif is smaller than the target size and cannot be cropped to 244x244
cropped_7.tif
Image at \Yehmh\FuShan\Fushan_seg_images\cropped